In [1]:
import asyncio
import psutil
import os
import time
import threading
# Importe suas funções de teste
# from test_pipeline import test_full_workflow 

# --- Utilitário de Formatação ---
def format_bytes(size):
    power = 2**10
    n = 0
    power_labels = {0 : '', 1: 'KB', 2: 'MB', 3: 'GB', 4: 'TB'}
    while size > power:
        size /= power
        n += 1
    return f"{size:.2f} {power_labels[n]}"

# --- Classe Monitor de Memória ---
class MemoryMonitor:
    def __init__(self, interval=0.1):
        self.interval = interval
        self.process = psutil.Process(os.getpid())
        self.running = False
        self.max_rss = 0
        self.start_rss = 0
        self.end_rss = 0
        self._thread = None

    def _monitor(self):
        while self.running:
            # RSS: Resident Set Size (Memória RAM física usada)
            current_rss = self.process.memory_info().rss
            if current_rss > self.max_rss:
                self.max_rss = current_rss
            time.sleep(self.interval)

    def start(self):
        self.start_rss = self.process.memory_info().rss
        self.max_rss = self.start_rss
        self.running = True
        self._thread = threading.Thread(target=self._monitor, daemon=True)
        self._thread.start()
        print(f"📉 Memória Inicial: {format_bytes(self.start_rss)}")

    def stop(self):
        self.running = False
        if self._thread:
            self._thread.join()
        self.end_rss = self.process.memory_info().rss
        
        print("\n" + "="*40)
        print(f"📊 RELATÓRIO DE MEMÓRIA DO PROCESSO")
        print("="*40)
        print(f"📉 Inicial:      {format_bytes(self.start_rss)}")
        print(f"📈 Final:        {format_bytes(self.end_rss)}")
        print(f"🚀 PICO (Peak):  {format_bytes(self.max_rss)}")
        print(f"💧 Diferença:    {format_bytes(self.end_rss - self.start_rss)}")
        print("="*40)


In [2]:
import asyncio
import pandas as pd
import psutil
from app.tools.context_store import AnalysisContext 
from deepagents import create_deep_agent
from app.tools.metrics_agent_tools import get_dataset_health_check, query_anomalous_ids, run_ml_inference_pipeline
from app.tools.data_tools import *

orchestrator_tools = [
      query_mongo_requests, 
      query_sql_campaigns, 
      list_avaiable_datasets, 
      inspect_file_schema, 
      load_dataset_into_context, 
      check_context_status, 
      inspect_file_schema
]


async def test_full_workflow():
    print("\n🔹 --- INICIANDO TESTE DE INTEGRAÇÃO (SEM LLM) ---")

    # ==============================================================================
    # 1. SIMULAÇÃO DO ORQUESTRADOR (Discovery & Ingestion)
    # ==============================================================================
    print("\nTesting: 1. Orchestrator - Discovery (SQL)")
    
    # Simula o LLM chamando a tool com argumentos
    # Nota: Se suas tools usam @tool, use .invoke() ou chame a função decorada diretamente dependendo da versão do LangChain
    try:
        # Tenta listar campanhas do Google
        campaigns_str = await query_sql_campaigns.ainvoke({"traffic_source": "google", "limit": 2})
        print(f"✅ SQL Result: {campaigns_str}")
        
        # HACK PARA O TESTE:
        # Como não temos o LLM para ler a string e escolher o hash, vamos pegar um hash 'fake' 
        # ou extrair da string se o banco estiver conectado.
        # Para este teste, vou assumir que você pegou um hash válido do log acima.
        target_hash = "uw0qfu4a1r" 
        print(f"👉 Usando Hash Alvo: {target_hash}")

    except Exception as e:
        print(f"❌ SQL Failed: {e}")
        return

    print("\nTesting: 2. Orchestrator - Ingestion (Mongo -> Context)")
    try:
        # Simula o carregamento

        status_msg = await query_mongo_requests.ainvoke({
            "hash": target_hash,
            # "hashes": campaigns_str, 
            "traffic_source": "google"
        })
        print(f"✅ Load Result: {status_msg}")
        
        # VERIFICAÇÃO DE ESTADO (Crucial!)
        # Vamos espiar dentro do Singleton para ver se funcionou
        try:
            print("Status: ", AnalysisContext.get_status())
            # print(f"🔎 VERIFICAÇÃO: Contexto contém {len(df)} linhas. Colunas: {list(df.columns[:3])}...")
        except ValueError:
            print("❌ VERIFICAÇÃO FALHOU: Contexto está vazio!")
            return

    except Exception as e:
        print(f"❌ Mongo Load Failed: {e}")
        # SE VOCÊ NÃO TEM BANCO RODANDO AGORA, DESCOMENTE A LINHA ABAIXO PARA MOCKAR DADOS:
        # mock_data_loading() 
        return

    # ==============================================================================
    # 2. SIMULAÇÃO DO SUB-AGENTE (Analysis & Inference)
    # ==============================================================================
    print("\nTesting: 3. Sub-agent - ML Execution")
    try:
        # O agente chama sem argumentos, pois pega do Contexto
        inference_summary = run_ml_inference_pipeline.invoke({}) 
        print(f"✅ Inference Result: {inference_summary}")
    except Exception as e:
        print(f"❌ ML Pipeline Failed: {e}")
        return

    print("\nTesting: 4. Sub-agent - Health Check")
    try:
        health_stats = get_dataset_health_check.invoke({})
        print(f"✅ Health Stats: {health_stats}")
    except Exception as e:
        print(f"❌ Health Check Failed: {e}")

    print("\nTesting: 5. Sub-agent - Query Anomalies")
    try:
        # Testa buscar IDs com baixa confiança
        anomalies = query_anomalous_ids.invoke({"criteria": "low_trust", "threshold": 0.5})
        print(f"✅ Found {len(anomalies)} anomalies. Sample IDs: {anomalies[:5]}")
    except Exception as e:
        print(f"❌ Query Failed: {e}")

    print("\n🔹 --- TESTE FINALIZADO ---")

# --- MOCK OPCIONAL (Se você não tiver o Mongo rodando localmente) ---
def mock_data_loading():
    print("⚠️ MOCKING DATA LOADING...")
    data = {
        "id": range(100),
        "user_agent": ["Mozilla/5.0"] * 50 + ["Googlebot"] * 50,
        "url": ["/home"] * 100,
        "label": [1]*50 + [0]*50 # 1=Human, 0=Bot
    }
    df = pd.DataFrame(data)
    AnalysisContext.set_mongo_data(df, "google")
    print("✅ Mock data loaded into Context.")

# --- Seu Wrapper de Teste ---
async def run_with_monitoring():
    monitor = MemoryMonitor(interval=0.1) # Checa a cada 100ms
    
    monitor.start()
    try:
        print("⏳ Executando Workflow...")
        # Chama sua função original aqui
        await test_full_workflow() 
    finally:
        monitor.stop()

# --- Se você usar PyTorch (GPU/VRAM) ---
def check_gpu_memory():
    try:
        import torch
        if torch.cuda.is_available():
            print("\n RELATÓRIO GPU (VRAM)")
            print(f"Alocada: {format_bytes(torch.cuda.memory_allocated())}")
            print(f"Reservada: {format_bytes(torch.cuda.memory_reserved())}")
    except ImportError:
        pass

if __name__ == "__main__":
    await run_with_monitoring()

c:\Users\Camille\Documents\TWR\deep_agents_twr\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Camille\Documents\TWR\deep_agents_twr\.venv\Lib\site-packages\motor\core.py:171: UserWarning: You appear to be connected to a DocumentDB cluster. For more information regarding feature compatibility and support please visit https://www.mongodb.com/supportability/documentdb
  delegate = self.__delegate_class__(*args, **kwargs)


Garantindo índices...
📉 Memória Inicial: 512.77 MB
⏳ Executando Workflow...

🔹 --- INICIANDO TESTE DE INTEGRAÇÃO (SEM LLM) ---

Testing: 1. Orchestrator - Discovery (SQL)
✅ SQL Result: ['wag1u37igk', 'imrawov20j']
👉 Usando Hash Alvo: uw0qfu4a1r

Testing: 2. Orchestrator - Ingestion (Mongo -> Context)
DEBUG [Context]: Mongo Data Loaded. Rows: 1000
✅ Load Result: SUCCESS: Loaded 1000 requests into AnalysisContext.
Sources: google | Hashes: 1
Action Required: Delegate to 'Metrics Analyst' agent to run ML inference now.
Status:  Mongo Raw: 1000 | ML Processed: Pending

Testing: 3. Sub-agent - ML Execution
                          _id                datetime decision  \
0    698e1efda265b281ec6cb12b 2026-02-12 18:42:05.331   unsafe   
1    698e1e8da265b281ec6c52fc 2026-02-12 18:40:13.101   unsafe   
2    698e1cffa265b281ec6b1ce8 2026-02-12 18:33:35.886     bots   
3    698e1cfca265b281ec6b1a25 2026-02-12 18:33:32.065     bots   
4    698e1c5ca265b281ec6ac3ee 2026-02-12 18:30:52.897   unsaf

Loading weights: 100%|██████████| 103/103 [00:00<00:00, 300.07it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
Batches: 100%|██████████| 16/16 [00:32<00:00,  2.06s/it]


DEBUG [Context]: ML Results Stored. Rows: 1000
checando se a tool de inferencia salva dos dados: 1000
✅ Inference Result: Inference completed using 'google' model with results: 
Models Accuracy: 0.277Total Error in prediction (possible anomalies): 723Analyzed 1000 samples.
You can now now:
1. Call 'get_dataset_health_check' to see overall performance stats.
2. Call 'query_anomalous_ids' to extract specific samples for the Detective Agent.

Testing: 4. Sub-agent - Health Check
✅ Health Stats: {'total_samples': 1000, 'false_positives': 1, 'false_negatives': 722, 'avg_trust': 0.3169092833995819}

Testing: 5. Sub-agent - Query Anomalies
❌ Query Failed: 'id'

🔹 --- TESTE FINALIZADO ---

📊 RELATÓRIO DE MEMÓRIA DO PROCESSO
📉 Inicial:      512.77 MB
📈 Final:        1.01 GB
🚀 PICO (Peak):  1.02 GB
💧 Diferença:    521.10 MB


In [3]:
from app.config.container import request_service
from app.config.container import campaign_service


campaigns = await campaign_service.fetch_recent_active_campaigns(traffic_source="google", limit=50)
results = await request_service.fetch_training_sample_by_hashes(campaigns)

CancelledError: 

In [ ]:
import pandas as pd

df_results = pd.DataFrame(results)
df_results["decision"].value_counts()

decision
bots      10000
unsafe    10000
Name: count, dtype: int64